In [1]:
%pwd

'/home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/research'

In [3]:
import os
os.chdir("../")

In [27]:
%pwd

'/home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [34]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [39]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH
from src.cnnClassifier.constants import PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [40]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [41]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """ 
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [42]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-05 23:02:51,127: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-05 23:02:51,135: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-05 23:02:51,160: INFO: common: created directory at: artifacts]
[2023-07-05 23:02:51,162: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-05 23:02:53,984: INFO: 2901002007: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7927736
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3326d88b4626011e21b259b99f4bd5720d018258a66a866f3245e49898a91d1e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A460:34B768:1D96D:27A10:64A5A93D
Accept-Ranges: bytes
Date: Wed, 05 Jul 2023 17:32:52 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4737-BOM
X-Cache: MISS